In [1]:
import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.connect('nba.db')
conn.execute("""ATTACH 'covid19.db' AS covid19""")
conn.execute("""ATTACH 'twElection2020.db' AS twElection2020""")
conn.execute("""ATTACH 'imdb.db' AS imdb""")

###### Overview
* 練習建立虛擬資料表
* 練習刪除虛擬資料表

## 從 covid19 資料庫建立一個虛擬資料表名為 total_confirmed_by_country_region 記錄截至 2021-03-31 全球各國的確診人數
* 練習建立虛擬資料表以及使用水平整合

In [3]:
create_view_total_confirmed_by_country_region_from_covid19 =\
"""
-- SQL 語法起點

CREATE VIEW covid19.total_confirmed_by_country_region (Country_Region, total_confirmed)
    AS
SELECT lookup_table.Country_Region,
       SUM(daily_report.Confirmed) AS total_confirmed
  FROM lookup_table
 JOIN daily_report
   ON lookup_table.Combined_Key = daily_report.Combined_Key
 GROUP BY lookup_table.Country_Region;

-- SQL 語法終點
"""
# view_total_confirmed_by_country_region_from_covid19 = pd.read_sql(create_view_total_confirmed_by_country_region_from_covid19, conn)
# view_total_confirmed_by_country_region_from_covid19

test_view = \
"""
-- SQL 語法起點

SELECT * 
  FROM total_confirmed_by_country_region;
  
-- SQL 語法終點  
"""

view = pd.read_sql(test_view, conn)
view


,Country_Region,total_confirmed
0,Afghanistan,56454
1,Albania,125157
2,Algeria,117192
3,Andorra,12010
4,Angola,22311
...,...,...
187,Vietnam,2603
188,West Bank and Gaza,242353
189,Yemen,4357
190,Zambia,88418


## 41. 從 twElection2020 資料庫建立一個虛擬資料表名為 presidential_total_votes 記錄三組候選人的總得票數

* 練習建立虛擬資料表以及水平整合練習

In [4]:
create_view_presidential_total_votes_from_twelection2020 =\
"""
-- SQL 語法起點

CREATE VIEW twElection2020.presidential_total_votes (number, candidate, total_votes)
    AS
SELECT candidates.id AS number,
       candidates.candidate AS candidate,
       SUM(presidential.votes) AS total_votes
  FROM presidential
 JOIN candidates
   ON candidates.id = presidential.candidate_id
 WHERE candidates.id IN (SELECT DISTINCT presidential.candidate_id FROM presidential)
 GROUP BY presidential.candidate_id;

-- SQL 語法終點
"""
# view_presidential_total_votes_from_twelection2020 = pd.read_sql(create_view_presidential_total_votes_from_twelection2020, conn)
# view_presidential_total_votes_from_twelection2020


test_view = \
"""
-- SQL 語法起點

SELECT * 
  FROM presidential_total_votes;

-- SQL 語法終點
"""

view = pd.read_sql(test_view, conn)
view


,number,candidate,total_votes
0,1,宋楚瑜/余湘,608590
1,2,韓國瑜/張善政,5522119
2,3,蔡英文/賴清德,8170231


## 42. 從 nba 資料庫建立一個虛擬資料表名為 ppg_leader_by_teams 紀錄各個球隊的得分王（生涯場均得分 ppg 全隊最高）是誰

In [5]:
create_view_ppg_leader_by_teams_from_nba =\
"""
-- SQL 語法起點

CREATE VIEW nba.ppg_leader_by_teams (team, firstName, lastName, ppg)
    AS
SELECT teams.fullName AS team,
       players.firstName,
       players.lastName,
       MAX(career_summaries.ppg) AS ppg
  FROM players
 JOIN teams
   ON teams.teamId = players.teamId
 JOIN career_summaries
   ON players.personId = career_summaries.personId
 GROUP BY players.teamId;

-- SQL 語法終點
"""

# view_ppg_leader_by_teams_from_nba = pd.read_sql(create_view_ppg_leader_by_teams_from_nba, conn)
# view_ppg_leader_by_teams_from_nba

test_view = \
"""
-- SQL 語法起點

SELECT * 
  FROM ppg_leader_by_teams;

-- SQL 語法終點
"""

view = pd.read_sql(test_view, conn)
view

,team,firstName,lastName,ppg
0,Atlanta Hawks,Trae,Young,24.0
1,Boston Celtics,Kemba,Walker,19.8
2,Cleveland Cavaliers,Collin,Sexton,19.7
3,New Orleans Pelicans,Zion,Williamson,25.0
4,Chicago Bulls,Zach,LaVine,18.8
5,Dallas Mavericks,Luka,Doncic,25.6
6,Denver Nuggets,Nikola,Jokic,18.0
7,Golden State Warriors,Stephen,Curry,23.8
8,Houston Rockets,John,Wall,19.0
9,LA Clippers,Paul,George,20.1
